# Tutorial 2 - SVM Regression

We will predict the price (`price` column) of an AirBNB listing in Boston given a number of features about the listing.

**Therefore, our unit of analysis is an AIRBNB LISTING**

# Setup

In [ ]:
# Common imports
import numpy as np
import pandas as pd

np.random.seed(42)


# Get the data

In [ ]:
#We will predict the "price" value in the data set:

airbnb = pd.read_csv("airbnb.csv")
airbnb.head()

# Split the data into train and test

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(airbnb, test_size=0.3)

### Be careful: we haven't seperated the target column yet

## Check the missing values

In [ ]:
train_set.isna().sum()

In [ ]:
test_set.isna().sum()

# Data Prep

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Drop the variables we can't use in this tutorial

In [ ]:
# We can't use the following columns in this tutorial, because they are for classification tasks

train = train_set.drop(['price_gte_150', 'price_category'], axis=1)
test = test_set.drop(['price_gte_150', 'price_category'], axis=1)

## Separate the target variable (we don't want to transform it)

In [ ]:
train_y = train[['price']]
test_y = test[['price']]

train_inputs = train.drop(['price'], axis=1)
test_inputs = test.drop(['price'], axis=1)

##  Identify the numerical and categorical columns

In [ ]:
train_inputs.dtypes

**At this stage, you can manually identify numeric, binary, and categorical columns as follows:**

`numeric_columns = ['latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'Number of amenities', 'guests_included', 'price_per_extra_person', 'minimum_nights', 'number_of_reviews', 'number_days_btw_first_last_review', 'review_scores_rating']`
 
 `binary_columns = ['host_is_superhost', 'host_identity_verified']`
 
 `categorical_columns = ['neighbourhood_cleansed', 'property_type', 'room_type', 'bed_type', 'cancellation_policy']`
 
<br>
 
**If you do not want to manually type these, you can do the below tricks:**

In [ ]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [ ]:
# Identify the binary columns so we can pass them through without transforming
binary_columns = ['host_is_superhost', 'host_identity_verified']

In [ ]:
# Be careful: numerical columns already includes the binary columns,
# So, we need to remove the binary columns from numerical columns.

for col in binary_columns:
    numeric_columns.remove(col)

In [ ]:
binary_columns

In [ ]:
numeric_columns

In [ ]:
categorical_columns

# Pipeline

In [ ]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='mean')),
                ('scaler', StandardScaler())])

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [ ]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [ ]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

In [ ]:
train_x.shape

# Tranform: transform() for TEST

In [ ]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

In [ ]:
test_x.shape

# Calculate the baseline

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
#First find the average value of the target

mean_value = np.mean(train_y['price'])

mean_value

In [ ]:
# Predict all values as the mean

baseline_pred = np.repeat(mean_value, len(test_y))

baseline_pred

In [ ]:
baseline_mse = mean_squared_error(test_y, baseline_pred)

baseline_rmse = np.sqrt(baseline_mse)

print('Baseline RMSE: {}' .format(baseline_rmse))

In [ ]:
train_y['price']

# LinearSVR

This is the support vector regressor. Preferred over `SVR(kernel='linear')`

In [ ]:
from sklearn.svm import LinearSVR 

svm_reg = LinearSVR(C=100, epsilon=0.5, max_iter=10000) 

svm_reg.fit(train_x, train_y)

In [ ]:
#Train RMSE
svm_train_pred = svm_reg.predict(train_x)

train_mse = mean_squared_error(train_y, svm_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

In [ ]:
#Test RMSE
svm_test_pred = svm_reg.predict(test_x)

test_mse = mean_squared_error(test_y, svm_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

# SVR(kernel='poly')

In [ ]:
from sklearn.svm import SVR 

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.5) 

svm_poly_reg.fit(train_x, train_y)

In [ ]:
#Train RMSE
svm_train_pred = svm_poly_reg.predict(train_x)

train_mse = mean_squared_error(train_y, svm_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

In [ ]:
#Test RMSE
svm_test_pred = svm_poly_reg.predict(test_x)

test_mse = mean_squared_error(test_y, svm_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

# SVR(kernel='rbf')

In [ ]:
#Takes too long to converge

svm_rbf_reg = SVR(kernel="rbf", C=100, epsilon=0.01, gamma='scale') 

svm_rbf_reg.fit(train_x, train_y)

In [ ]:
#Train RMSE
svm_train_pred = svm_rbf_reg.predict(train_x)

train_mse = mean_squared_error(train_y, svm_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

In [ ]:
#Test RMSE
svm_test_pred = svm_rbf_reg.predict(test_x)

test_mse = mean_squared_error(test_y, svm_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

# Cross Validation

We perform k-fold cross-validation.<br>
See this link for more info: https://scikit-learn.org/stable/modules/cross_validation.html

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
scores = cross_validate(svm_rbf_reg, train_x, 
                        train_y, cv=5, scoring="neg_mean_squared_error",
                        return_estimator=True)

In [ ]:
scores

In [ ]:
# test scores are negative and squared

scores['test_score']

In [ ]:
rmse_scores = np.sqrt(-scores['test_score'])

rmse_scores

In [ ]:
rmse_scores.mean(), rmse_scores.std()

In [ ]:
# How can you make predictions: Use each estimator...

test_predictions = pd.DataFrame()

test_predictions['estimator_0'] = scores['estimator'][0].predict(test_x)
test_predictions['estimator_1'] = scores['estimator'][1].predict(test_x)
test_predictions['estimator_2'] = scores['estimator'][2].predict(test_x)
test_predictions['estimator_3'] = scores['estimator'][3].predict(test_x)
test_predictions['estimator_4'] = scores['estimator'][4].predict(test_x)

In [ ]:
test_predictions

In [ ]:
# Take the average of all predictions:

test_predictions['avg_prediction'] = np.mean(test_predictions, axis=1)

test_predictions

In [ ]:
# Test RMSE

test_mse = mean_squared_error(test_y, test_predictions['avg_prediction'])

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))